### Runnable 
* Runnable 은 런타임에 실행될 수 있는 모든 객체를 의미한다.

### Runnable 종류
* invoke : 입력에 대해 체인을 호출한다.
* stream : 응답의 청크를 스트리밍한다.
* batch : 입력 목록에 대한 일괄 처리를 수행한다.

### 이외에 비동기 메소드도 존재한다.
* astream : 비동기적으로 응답의 청크를 비동기 스트리밍한다.
* ainvoke : 비동기적으로 입력에 대해 체인을 호출한다.
* abatch : 비동기적으로 입력 목록에 대해 일괄 처리를 수행한다.
* astream_log : 최종 응답 및 발생하는 중간단계를 스트리밍한다.

In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path='../.env')

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name = 'gpt-4o-mini',
    temperature=0.1
)

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("{lecture}에 대해 3문장으로 설명해줘")

chain = prompt | llm | StrOutputParser()

In [ ]:
print(chain.invoke({"lecture":"임진왜란"}))

### stream [실시간츨력]

* 데이터 스트림을 생성하고, 스트림을 반복하여 각 데이터의 내용을 즉시 출력한다.
* end="" 인자를 사용해 줄바꿈을 하지 않을 수 있다.
* flush=True 인자를 사용해 즉시 출력 가능

In [ ]:
for token in chain.stream({"lecture":"고종황제"}):
    print(token,end="",flush=True)

### batch [단위실행]
* 여러개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각각의 입력에 대한 체인을 실행한다.
* max_cocurrency : 최대 병렬 처리 수리르 지정할 수 있다.

In [ ]:
result = chain.batch(
    [
        {"lecture":"이순신"},
        {"lecture":"조선선조"},
        {"lecture":"임진왜란"},       
    ],
    config={"max_concurrency":3}
)

print(result)

---
Parallel (병렬성)
* LCEL를 사용하여 체인을 구성할때, 여러 체인을 동시에 실행 할 수 있다.


In [10]:
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | llm 
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country} 의 수장은 누구야?")
    | llm 
    | StrOutputParser()
)

In [11]:
from langchain_core.runnables import RunnableParallel

# 병렬 실행 체인
combined = RunnableParallel(capital= chain1, area=chain2)

In [ ]:
print(chain1.invoke({"country":"프로이센"}))

In [ ]:
print(chain2.invoke({"country":"독일제국"}))

In [ ]:
result = combined.invoke({'country':'독일제국'})
print(result)

In [ ]:
print(result["capital"])
print(result["area"])

### RunnablePassThrough
* 입력을 그대로 전달하는 역할
* 단독으로 호출될 경우 단순히 입력을 받아 그대로 전달

In [18]:
llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.0
)

prompt = PromptTemplate.from_template("{num}의 약수를 알려줘")

chain = prompt | llm | StrOutputParser()

In [19]:
# invoke()를 통해 실행할때는 입력이 딕셔너리여야 하지만,
# 1개의 변수만 템플릿에 작성이 되었다면, 값만 전달해도 된다.
chain.invoke(10)

'10의 약수는 1, 2, 5, 10입니다.'

In [22]:
from langchain_core.runnables import RunnablePassthrough

passthrough_chain = {'num': RunnablePassthrough()} | prompt | llm | StrOutputParser()

passthrough_chain.invoke({'num':10})

'10의 약수는 1, 2, 5, 10입니다.'

** RunnablePassthrough.assign()**
* 입력값으로 들어온 값의 key/value 쌍을 새롭게 할당된 key/value 쌍을 합쳐준다.

In [23]:
RunnablePassthrough.assign(new_date=lambda x:x['num']*3).invoke({'num':1})

{'num': 1, 'new_date': 3}

### RunnbaleLambda
* 입력값에 대한 추가 처리를 수행할 수 있다.
* 람다 함수를 사용해 입력 값에 대한 추가 처리를 진행
* 날씨, 시간

In [34]:
from datetime import datetime

def get_today(num):
    # print("num : ", num)
    return datetime.now().strftime("%b-%d")

# get_today()

In [39]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter


# prompt
prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n}명을 알려줘"
)

# llm
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# chain 생성

chain = (
    {"today": RunnableLambda(get_today), "n": itemgetter("n")}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
print(chain.invoke({"n":3}))
